In [1]:
import os
import pandas as pd
import panel as pn
import altair as alt
from IPython.display import display

from toolz.curried import pipe

def json_dir(data, data_dir='altairdata'):
    os.makedirs(data_dir, exist_ok=True)
    return pipe(data, alt.to_json(filename=data_dir + '/{prefix}-{hash}.{extension}') )


alt.renderers.enable('default')
pn.extension('vega')

alt.data_transformers.register('json_dir', json_dir)
alt.data_transformers.enable('json_dir', data_dir='mydata')


DataTransformerRegistry.enable('json_dir')

In [9]:
# Lendo os dados do arquivo XLSX
file_path = '../../scrapper-dataset/excel/all-companies.xlsx'

data = pd.read_excel(file_path)
data['Data_only'] = data['data_criacao'].dt.date


from tools.problemaAgrupador import realiza_agrupamento as ra1
from tools.produtoAgrupador import realiza_agrupamento as ra2
from tools.categoriaAgrupador import realiza_agrupamento as ra3

ra1(data)
ra2(data)
ra3(data)

In [3]:
data.sort_values('sla', ascending=False).head()

,id,titulo,status,data_criacao,reclamacao,estado,cidade,categoria,problema,produto,voltaria_fazer_negocio,nota,respondido_em,sla,problema_agrupado,produto_agrupado,categoria_agrupado
49590,24041997,Fidelidade mercado livre,Respondida,2017-02-07 10:59:00,BOM DIA! TENHO UM CARTÃO MERCADO LIVRE E QUERO...,MG,Belo Horizonte,Cartões de Crédito,Qualidade do serviço prestado,Cartão de crédito,False,NaN,2020-02-17 11:28:00,95473740.0,Qualidade do Produto/Serviço,Outros,Compra e Venda
52610,24218481,Paguei frete Sedex e despacharam via PAC.,Respondida,2017-02-13 11:54:00,Comprei um produto no mercado livre e optei pe...,BA,Salvador,Problemas com o Site,Divergência de preço,Pagamentos e Documentos,False,NaN,2020-02-17 11:43:00,94952940.0,Outros,Outros,Outros
52609,24232669,Paguei mais caro do que o anuncio,Respondida,2017-02-13 19:19:00,Fiz contato com um vededor para saber se tinha...,MG,Ipatinga,Problemas com o Site,Divergência de preço,Pagamentos e Documentos,False,NaN,2020-02-17 11:43:00,94926240.0,Outros,Outros,Outros
49062,24409181,PROGRAMA DE FIDELIDADE,Respondida,2017-02-21 14:35:00,NÃO CONSIGO UTILIZAR MEUS PONTOS DE FIDELIDADE...,MG,Montes Claros,Problemas com o Site,Promoções,Propaganda Enganosa,False,NaN,2020-02-17 11:57:00,94252920.0,Propaganda e Publicidade,Outros,Outros
52608,24770721,Valor diferente do frete,Respondida,2017-03-10 21:52:00,"Na última compra que fiz no ML, o valor do fre...",ES,Cariacica,Problemas com o Site,Divergência de preço,Pagamentos e Documentos,False,NaN,2020-02-17 16:25:00,92773980.0,Outros,Outros,Outros


In [4]:
df_media_resolucao_cat = data.groupby('categoria_agrupado')['sla'].mean().reset_index()
df_media_resolucao_cat = df_media_resolucao_cat.sort_values('sla', ascending=False)
df_top_10_cat = df_media_resolucao_cat.head(10)
df_top_10_cat['sla'] = df_top_10_cat['sla'] / 60 / 60 / 24 # Converter para dias

df_media_resolucao_prob = data.groupby('problema_agrupado')['sla'].mean().reset_index()
df_media_resolucao_prob = df_media_resolucao_prob.sort_values('sla', ascending=False)
df_top_10_prob = df_media_resolucao_prob.head(10)
df_top_10_prob['sla'] = df_top_10_prob['sla'] / 60 / 60 / 24 # Converter para dias


df_media_resolucao_prod = data.groupby('produto_agrupado')['sla'].mean().reset_index()
df_media_resolucao_prod = df_media_resolucao_prod.sort_values('sla', ascending=False)
df_top_10_prod = df_media_resolucao_prod.head(10)
df_top_10_prod['sla'] = df_top_10_prod['sla'] / 60 / 60 / 24 # Converter para dias


C:\Users\thiag\AppData\Local\Temp\ipykernel_12144\998355324.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_top_10_prod['sla'] = df_top_10_prod['sla'] / 60 / 60 / 24 # Converter para dias


In [10]:
selection = alt.selection_interval(encodings=['x'])

chart = alt.Chart(data).mark_line().encode(
    x=alt.X('Data_only:T', title='Data'),
    y=alt.Y('count():Q', title='Quantidade de Reclamações'),
    color='status:N',
    tooltip=['Data_only:T', 'status:N', 'count()']
).add_selection(selection).properties(
    width=600,
    height=400,
    title='Quantidade de Reclamações ao Longo do Tempo por Status',
).interactive()


chart

alt.Chart(...)

In [6]:
chart_cat = alt.Chart(df_top_10_cat).mark_bar().encode(
    x=alt.X('categoria_agrupado:N', title='categoria', sort=None),
    y=alt.Y('sla:Q', title='Tempo Médio de Resolução (dias)'),
    color=alt.Color('categoria_agrupado:N', legend=None),
    tooltip=['categoria_agrupado:N', alt.Tooltip('sla:Q', format='.2f')]
).properties(
    width=600,
    title='Tempo Médio de Resolução por Categoria'
)

chart_prob = alt.Chart(df_top_10_prob).mark_bar().encode(
    x=alt.X('problema_agrupado:N', title='problema', sort=None),
    y=alt.Y('sla:Q', title='Tempo Médio de Resolução (dias)'),
    color=alt.Color('problema_agrupado:N', legend=None),
    tooltip=['problema_agrupado:N', alt.Tooltip('sla:Q', format='.2f')]
).properties(
    width=600,
    title='Tempo Médio de Resolução por Problema'
)


chart_prod = alt.Chart(df_top_10_prod).mark_bar().encode(
    x=alt.X('produto_agrupado:N', title='produto', sort=None),
    y=alt.Y('sla:Q', title='Tempo Médio de Resolução (dias)'),
    color=alt.Color('produto_agrupado:N', legend=None),
    tooltip=['produto_agrupado:N', alt.Tooltip('sla:Q', format='.2f')]
).properties(
    width=600,
    title='Tempo Médio de Resolução por Produto'
)
chart_cat | chart_prob | chart_prod

alt.HConcatChart(...)